In [1]:
import numpy as np
import matplotlib.pyplot
from qdots_qll.models import game

Parameter 0:  
True value: 0.3142159879207611 


Mean estimate: 0.32137 pm 0.00151
Median estimate: 0.32254 pm 0.00207
---
0.95 Confidence Interval for mean: 0.31849, 0.32428 
0.95 Confidence Interval for median: 0.31806, 0.32616 


Parameter 1:  
True value: 0.35833001136779785 


Mean estimate: 0.36799 pm 0.00169
Median estimate: 0.36649 pm 0.00218
---
0.95 Confidence Interval for mean: 0.36468, 0.37130 
0.95 Confidence Interval for median: 0.36207, 0.37061 


Parameter 2:  
True value: 0.05385100096464157 


Mean estimate: 0.05970 pm 0.00137
Median estimate: 0.05829 pm 0.00190
---
0.95 Confidence Interval for mean: 0.05700, 0.06238 
0.95 Confidence Interval for median: 0.05503, 0.06249 


Parameter 3:  
True value: -0.3336949944496155 


Mean estimate: -0.32731 pm 0.00222
Median estimate: -0.32609 pm 0.00289
---
0.95 Confidence Interval for mean: -0.33170, -0.32299 
0.95 Confidence Interval for median: -0.33183, -0.32051 

In [4]:
gn, gp, Sn, Sp =list(game.true_pars)

In [2]:
game.true_pars

Array([ 0.314216,  0.35833 ,  0.053851, -0.333695], dtype=float32)

# Estimated ones

In [3]:
gn, gp, Sn, Sp = [0.31473 / 2, 0.35860 / 2, 0.05631, -0.33043]

In [4]:
Gamma = 0.25 * (gp + gn)
zeta = 0.5 * (Sp + Sn)

In [5]:
delta = 0.12739334807998307
bigOmega = 0.5

T = 30
kbp = 0.130920339126989

eta = np.sqrt(delta**2 + bigOmega**2)
tanh = np.tanh(eta / (kbp * T * 2))

In [6]:
import scipy

In [7]:
f_aux = (
    lambda omegac: -np.exp((eta / omegac) ** 2)
    / 4
    * np.sqrt(np.pi)
    * (
        2 * eta**2 * omegac
        + omegac**3
        - 4 * eta**3 * scipy.special.dawsn(eta / omegac)
    )
)

In [8]:
f_to_solve = (
    lambda omegac: f_aux(omegac) - zeta * np.pi / 2 * eta**3 / tanh / Gamma
)

In [9]:
f_alpha_est = (
    lambda omega_c: (
        Gamma * 2 / np.pi / eta**3 * np.exp((eta / omega_c) ** 2)
    )
    * tanh
)

In [10]:
omega_c_hat = scipy.optimize.fsolve(f_to_solve, 1)
print(omega_c_hat)

alpha_hat = f_alpha_est(omega_c_hat)
print(alpha_hat)

[2.18247795]
[0.02705542]


In [13]:
n_occ = lambda omega: 1 / (np.exp(omega / kbp / T) - 1)
alpha = 0.027
wc = 2.2
alpha * eta**3 * np.exp(-((eta / wc) ** 2)) * np.pi / 2 / tanh

0.08406837396479773

# Estimation of error using fisher information

In [41]:
def error_omega_c_hat(omega_c_hat, deltazeta, deltaGamma):
    return np.sqrt(
        1
        / tanh**2
        * np.pi
        * eta**6
        * omega_c_hat**4
        * np.exp(-2 * ((eta / omega_c_hat) ** 2))
        * ((deltazeta) ** 2 + deltaGamma**2 * Gamma**2 * zeta**2)
        / (
            Gamma
            * 2
            * (
                -(eta**2) * omega_c_hat
                + omega_c_hat**3
                + 2 * eta**3 * scipy.special.dawsn(eta / omega_c_hat)
            )
        )
    )

In [42]:
estimated_ones =  [0.32137 , 0.36799 , 0.05970, -0.32731]

In [43]:
def errorGamma(deltagp, deltagm):
    return 0.25 * np.sqrt(deltagp**2 + deltagm**2)


def errorZeta(deltaSp, deltaSm):
    return 0.5 * np.sqrt(deltaSp**2 + deltaSm**2)

In [44]:
deltagm, deltagp, deltaSm, deltaSp = [
    abs(0.31406 - 0.31538) / 2,
    0.5 * abs(0.35794 - 0.35929),
    0.5 * abs(0.05541 - 0.05720),
    0.5 * abs(-0.33190 - -0.32896),
]

In [45]:
deltaGamma = errorGamma(deltagp, deltagm)
deltaZeta = errorZeta(deltaSp, deltaSm)

In [46]:
deltaZeta

0.0008605122021215035

In [47]:
delta_omega_c = error_omega_c_hat(omega_c_hat, deltaZeta, deltaGamma)

In [48]:
def error_alpha(omega_c_hat, deltaomegac, deltaGamma):
    return np.sqrt(
        (
            4
            * np.exp(2 * ((eta / omega_c_hat) ** 2))
            * (
                4 * deltaomegac**2 * Gamma**2 * eta**4
                + deltaGamma**2 * omega_c_hat**6
            )
        )
        / (1 / tanh**2 * np.pi**2 * eta**6 * omega_c_hat**6)
    )


# (tanh * 4 * Gamma * np.exp(((eta / omega_c_hat) ** 2))) / ()
# return np.sqrt(
#     1
#     / tanh**2
#     * np.pi
#     * eta**6
#     * omega_c_hat**4
#     * np.exp(-2 * ((eta / omega_c_hat) ** 2))
#     * ((deltazeta) ** 2 + deltaGamma**2 * Gamma**2 * zeta**2)
#     / (
#         Gamma
#         * 2
#         * (
#             -(eta**2) * omega_c_hat
#             + omega_c_hat**3
#             + 2 * eta**3 * scipy.special.dawsn(eta / omega_c_hat)
#         )
#     )
# )

In [55]:
delta_alpha = error_alpha(omega_c_hat, delta_omega_c, deltaGamma)

In [50]:
np.round(7.7426228e-05, 4)

0.0001

In [52]:
print(f"{omega_c_hat} pm  {delta_omega_c}")

[2.18247795] pm  [0.0111583]


In [56]:
print(f"{alpha_hat} pm  {delta_alpha}")

[0.02705542] pm  [7.7426228e-05]


## Rounded:

In [65]:

print("Estimated omega_c = 2.18 pm  0.02")
print("True omega_c = 2.2")

print( "---")

print("Estimated alpha = 0.0271 pm  0.0001")
print("True alpha = 0.027")

Estimated omega_c = 2.18 pm  0.02
True omega_c = 0.2
---
Estimated alpha = 0.0271 pm  0.0001
True alpha = 0.027
